In [1]:
import json
import os
import typing as tp
from datetime import date, datetime

import numpy as np
import optuna
import pandas as pd
# from common_metrics.metrics.recsys import MAP, HitRate, NDCG, PrecisionRecall
from loguru import logger
from scipy import sparse
from sklearn.preprocessing import normalize
from tqdm import tqdm

In [76]:
# train1 = train1[train1.rating >= 3]

In [92]:
train1.head()

,userid,movieid,rating,timestamp
0,0,120,5.0,838985046
1,0,183,5.0,838983525
2,0,228,5.0,838983392
3,0,289,5.0,838983421
4,0,313,5.0,838983392


In [93]:
test1.head()

,userid,movieid,rating,timestamp
0,0,581,3.0,1208670024
1,0,9753,5.0,1208669800
2,0,9435,2.0,1208669698
3,0,9392,5.0,1208669977
4,0,9391,0.5,1208669967


In [94]:
holdout1.head()

,userid,movieid,rating,timestamp
0,0,6157,3.0,1208670181
1,1,4448,4.0,1199221172
2,2,9503,4.5,1188713380
3,3,5907,3.0,1190232446
4,4,7635,2.5,1192190218


In [95]:
items = train1.movieid.unique()
test1 = test1[test1.movieid.isin(items)]
users_test = test1.userid.unique()
holdout1 = holdout1[holdout1.userid.isin(users_test)]

In [96]:
def load_train_data(
        train_data: pd.DataFrame
) -> sparse.csr_matrix:
    """
    Creates csr_matrix for train
    """
    
    n_items = max(train_data.movieid) + 1
    n_users = max(train_data.userid) + 1
    rows, cols = train_data["userid"], train_data["movieid"]
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype="float64", shape=(n_users, n_items))
    return data

In [97]:
# n_items = max(max(train1.movieid), max(test1.movieid)) + 1
# n_users = max(max(train1.userid), max(test1.userid)) + 1

data = load_train_data(train1)
test = load_train_data(test1)

logger.info("data ready")

2022-07-08 21:20:01.978 | INFO     | __main__:<cell line: 7>:7 - data ready


In [98]:
assert data.shape[1] == test.shape[1]

In [99]:
def tune(
     train: sparse.csr_matrix, test: np.ndarray, holdout: pd.DataFrame,
    n_trials_ease: int = 25
) -> tp.Tuple[np.matrix, int]:
    logger.info("start tuning")

    def objective(trial):
        params_for_tune = {
            "reg_weight": trial.suggest_int("reg_weight", low=1, high=500)
        }
    
        item_matrix = fit_ease(train, params_for_tune["reg_weight"])
        scores = predict(test.toarray(), item_matrix)
        metric = HR(scores, holdout)
        return metric

    study = optuna.create_study(study_name="ease", direction="maximize")
    study.optimize(objective, n_trials=n_trials_ease, n_jobs=1)
    logger.info("end tuning")
    trial = study.best_trial
    reg_weight = trial.params['reg_weight']
    item_matrix = fit_ease(train, reg_weight)
    return item_matrix, reg_weight

In [100]:
def fit_ease(
    train: sparse.csr_matrix, reg_weight: int
) -> np.ndarray:
    """
    Counts item_matrix for future predictions
    """
    logger.info("start fitting")
    X = train
#     if cfg.norm:
#     logger.info("normalize")
#     X = normalize(X, norm="l2", axis=1)
#     X = normalize(X, norm="l2", axis=0)
#     X = sparse.csr_matrix(X)
    # gram matrix
    logger.info("gram matrix")
    G = X.T @ X
    # add reg to diagonal
    G += reg_weight * sparse.identity(G.shape[0])
    # convert to dense because inverse will be dense
    G = G.todense()
    # invert. this takes most of the time
    logger.info("invert")
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    # zero out diag
    np.fill_diagonal(B, 0.0)
    item_matrix = B
    return item_matrix

In [101]:
def predict(
        data: np.ndarray, item_matrix: np.ndarray, remove_seen: bool = True
) -> np.ndarray:
    """
    Counts scores
    """
    scores = data.dot(item_matrix)
    if remove_seen:
        scores[data > 0] = -1e13
    return scores

In [102]:
# item_matrix = fit_ease(data, reg_weight=367)
# scores = predict(test.toarray(), item_matrix)

# with tuning
item_matrix, reg_weight = tune(data, test, holdout1)
scores = predict(test.toarray(), item_matrix)

2022-07-08 21:20:04.200 | INFO     | __main__:tune:5 - start tuning
[I 2022-07-08 21:20:04,202] A new study created in memory with name: ease
2022-07-08 21:20:04.204 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:20:04.205 | INFO     | __main__:fit_ease:15 - gram matrix
2022-07-08 21:20:13.409 | INFO     | __main__:fit_ease:22 - invert
[I 2022-07-08 21:20:40,512] Trial 0 finished with value: 0.09084084084084085 and parameters: {'reg_weight': 102}. Best is trial 0 with value: 0.09084084084084085.
2022-07-08 21:20:40.513 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:20:40.514 | INFO     | __main__:fit_ease:15 - gram matrix
2022-07-08 21:20:50.833 | INFO     | __main__:fit_ease:22 - invert
[I 2022-07-08 21:21:23,704] Trial 1 finished with value: 0.09284284284284285 and parameters: {'reg_weight': 263}. Best is trial 1 with value: 0.09284284284284285.
2022-07-08 21:21:23.705 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:21:23.706 | INFO     

2022-07-08 21:38:52.925 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:38:52.928 | INFO     | __main__:fit_ease:15 - gram matrix
2022-07-08 21:39:20.824 | INFO     | __main__:fit_ease:22 - invert
[I 2022-07-08 21:40:21,237] Trial 21 finished with value: 0.09334334334334335 and parameters: {'reg_weight': 386}. Best is trial 14 with value: 0.09384384384384384.
2022-07-08 21:40:21.254 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:40:21.256 | INFO     | __main__:fit_ease:15 - gram matrix
2022-07-08 21:40:43.280 | INFO     | __main__:fit_ease:22 - invert
[I 2022-07-08 21:41:37,552] Trial 22 finished with value: 0.09259259259259259 and parameters: {'reg_weight': 287}. Best is trial 14 with value: 0.09384384384384384.
2022-07-08 21:41:37.561 | INFO     | __main__:fit_ease:7 - start fitting
2022-07-08 21:41:37.562 | INFO     | __main__:fit_ease:15 - gram matrix
2022-07-08 21:41:51.568 | INFO     | __main__:fit_ease:22 - invert
[I 2022-07-08 21:42:38,830] Trial 

In [103]:
def HR(scores, holdout1, count_of_rec = 10):
    count_of_true_rec = 0
    pred_array = np.argsort(-scores)[:, :count_of_rec]
    for index, row in holdout1.iterrows():
        movie = row.movieid
#         print(movie)
        recommend = pred_array[index]
        if movie in recommend:
            count_of_true_rec += 1
    return count_of_true_rec / len(holdout1)

In [104]:
hr = HR(scores, holdout1)
hr

0.09384384384384384

In [23]:
# def get_recommend(
#     scores: np.matrix,
#     count_of_rec: int = 10
# ) -> None:
#     """
#     Writes recommendations in json file
#     """
#     logger.info("start creating recommend")
#     scores = np.squeeze(np.asarray(scores))
#     recommend = {}

#     list_rec = np.argsort(-scores)[:, :count_of_rec]

#     users = test1.userid.unique()
#     items = list_rec

# #     for i in range(len(list_rec)):
# #         items.append([*map(id2item.get, list(list_rec[i]))])
#     recommend = dict(zip(users, items))
# #     path_to_data = os.path.join(cfg.json_path, "ease")
# #     if not os.path.exists(path_to_data):
# #         os.makedirs(path_to_data)
# #     with open(os.path.join(path_to_data, "recommend.json"), "w", encoding="utf-8") as f:
# #         json.dump(recommend, f, ensure_ascii=False, indent=2)
# #     logger.info("recommendations are ready")
#     return recommend

In [24]:
# recommend = get_recommend(scores)